In [2]:
%load_ext autoreload
%autoreload 2
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import pandas as pd
import os
import sys
import numpy as np
sys.path.append("../../Code")
import utils
import files as f
import seaborn as sns
import matplotlib.pyplot as plt
import preprocessing as pre
import languages as lang
stopWords = set(stopwords.words('english'))
import profiles
import afinns as af

/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Load Guests, Hosts, Reviews , and reverse reviews 

In [125]:
guests = pd.read_csv(os.path.join('..','2','nr_restrict_guests.csv'), low_memory=False, index_col=0).drop_duplicates(subset="id", keep="last")
hosts = pd.read_csv(os.path.join('..','2','nr_restrict_hosts.csv'), index_col=0)
hosts = hosts[~hosts.index.isnull()]
reviews = pd.read_csv(os.path.join('..','2','nr_restrict_reviews.csv'), index_col=0)

In [129]:
len(guests)

96255

In [130]:
len(hosts)

6924

## Get combined Reviews

### Guests

In [132]:
guests_full = pd.read_csv('../5/guests_outward_review_full.csv', index_col=0)

### Hosts

In [134]:
hosts_full = pd.read_csv('../5/hosts_outward_review_full.csv', index_col=0)

## Get Airbnb Dictionary

In [3]:
abnbvocab =  pd.read_csv(os.path.join('..','..', 'Tools','mydict.tsv'), delimiter="\t", index_col=0)

In [5]:
list(abnbvocab.index)

['information',
 'recommendations',
 'tips',
 'advice',
 'suggestions',
 'directions',
 'instructions',
 'useful',
 'details',
 'info',
 'explained',
 'communication',
 'questions',
 'quick',
 'quickly',
 'communicative',
 'communicate',
 'respond',
 'responded',
 'answer',
 'answered',
 'prompt',
 'communicated',
 'text',
 'request',
 'question',
 'promptly',
 'communicating',
 'requests',
 'communications',
 'answering',
 'emails',
 'response',
 'responding',
 'communicator',
 'responses',
 'concerns',
 'responds',
 'replied',
 'reply',
 'queries',
 'texts',
 'inquiries',
 'answers',
 'check',
 'provided',
 'arrival',
 'arrived',
 'late',
 'offered',
 'keys',
 'hours',
 'key',
 'flight',
 'prepared',
 'reception',
 'checking',
 'checked',
 'departure',
 'waiting',
 'arriving',
 'timely',
 'punctual',
 'checkout',
 'checkin',
 'punctuality',
 'place',
 'apartment',
 'clean',
 'house',
 'room',
 'space',
 'flat',
 'property',
 'care',
 'tidy',
 'neat',
 'spotless',
 'immaculate',
 'con

In [136]:
words = {}
for k in abnbvocab.category.unique():
    words[k] = set(abnbvocab[abnbvocab.category == k].index)

In [137]:
len(words['propEst']) + len(words['propFac']) + len(words['propInt']) + len(words['loct']) + len(words['condAct']) + len(words['condCom']) + len(words['condLog']) + len(words['condPrp']) + len(words['condPrs'])

305

In [138]:
len(words['sociMea']) + len(words['sociPpl']) + len(words['sociPrs']) + len(words['sociShr']) + len(words['sociTlk'])

68

In [139]:
def get_airbnb_counts(text, words):
    t = " ".join(eval(text)).lower()
    token_t = utils.tokenize(t)
    fin_t = " " + " ".join(token_t) + " "
    counts = {}
    result = {}
    for k in words.keys():
        count = 0
        for w in k:
            count += fin_t.count(" " + w + " ")
        counts[k] = count #/len(token_t) if len(token_t) != 0 else 0
    business_count = (counts['propEst'] + counts['propFac'] + counts['propInt'] + counts['loct'] + counts['condAct'] + counts['condCom'] + counts['condLog'] + counts['condPrp'] + counts['condPrs'])
    social_count = (counts['sociMea'] + counts['sociPpl'] + counts['sociPrs'] + counts['sociShr'] + counts['sociTlk'])
    combined_counts = business_count + social_count
    result['social_motivation'] = social_count/combined_counts if combined_counts != 0 else 0
    return result

In [140]:
airbnb_dict_h = [get_airbnb_counts(c, words) if len(c) > 0 else {k:np.nan for k in words.keys()} for c in hosts_full['review_text']]

In [141]:
airbnb_dict_g = [get_airbnb_counts(c, words) if len(c) > 0 else {k:np.nan for k in words.keys()} for c in guests_full['review_text']]

In [143]:
pd.options.display.max_colwidth = 100

In [144]:
airbnb_g = pd.DataFrame(airbnb_dict_g)

In [145]:
airbnb_h = pd.DataFrame(airbnb_dict_h)

In [146]:
airbnb_g.head()

,social_motivation
0,1.000000
1,0.903226
2,0.500000
3,0.500000
4,0.500000


In [147]:
airbnb_h['id'] = hosts_full['id']
airbnb_g['id'] = guests_full['id']

In [148]:
reviews = reviews.join(airbnb_g.rename(columns={'social_motivation':'guest_social_motivation'}).set_index('id'), on='reviewer_id')

In [149]:
reviews = reviews.join(airbnb_h.rename(columns={'social_motivation':'host_social_motivation'}).set_index('id'), on='recipient_id')

In [151]:
reviews[reviews.guest_social_motivation.isnull()]

,idGuestReview,id,date,reviewer_id,listing_id,recipient_id,comments,hostCancelled,token_len,num_of_sents,google_langs,google_langs_conf,langdetect_langs,langdetect_langs_conf,other_langs,translated,guest_social_motivation,host_social_motivation


In [152]:
reviews['social_motivation_hg_diff'] = reviews['host_social_motivation'] - reviews['guest_social_motivation']

In [153]:
reviews['social_motivation_abs_diff'] = np.abs(reviews['social_motivation_hg_diff'])

In [154]:
len(reviews)

215960

In [158]:
reviews[['id','date','reviewer_id','listing_id','recipient_id', 'social_motivation_hg_diff', 'social_motivation_abs_diff']].to_csv('airbnb_vocab_final.csv')

In [16]:
pd.options.display.max_colwidth = 200